In [10]:
import numpy as np
import matplotlib.pyplot as plt
from numpy import sqrt 

In [11]:
#utils
def Sqr(a):
    return a*a
 
def Distance(x1,y1,x2,y2):
    return sqrt(Sqr(y2-y1)+Sqr(x2-x1))

In [12]:
fileToOpen = "p01.txt"

m = 0
n = 0
t = 0

In [20]:
ct = 0
with open(fileToOpen) as fin:
    rows = (line.split() for line in fin)
    for row in rows:
        if ct == 0:
            m = int(row[0])
            n = int(row[1])
            t = int(row[2])
            tmp = np.zeros([int(t),2])
            depotz = np.zeros([t, 5])
            customerz = np.zeros([n,5])
        elif ct > 0 and ct <= t:
            for depot in tmp:
                tmp[ct-1] = (row[0],row[1])
        elif ct >= t+1 and ct <= n+t:
            for element in range(n):
                a = n+t+1
                customerz[ct-a] = (row[0],row[1],row[2],row[3],row[4])
        elif ct > t+n:
            a = n+t+1
            for element in range(t):
                mdr = tmp[element][0]
                mlv = tmp[element][1]
                depotz[ct-a] = (row[0],row[1],row[2],mdr,mlv)
        ct = ct + 1
                
print (str(m) +" "+ str(n) +" "+ str(t) )

vehiclesMxW = depotz[0][4]
nbVehicles = 16 #np.random.randint(t, m*t)  #aka number of routes
indivPerGen = 100

4 50 4


In [26]:

print(depotz)
print(customerz)

[[51. 20. 20.  0. 80.]
 [52. 30. 40.  0. 80.]
 [53. 50. 30.  0. 80.]
 [54. 60. 50.  0. 80.]]
[[ 1. 37. 52.  0.  7.]
 [ 2. 49. 49.  0. 30.]
 [ 3. 52. 64.  0. 16.]
 [ 4. 20. 26.  0.  9.]
 [ 5. 40. 30.  0. 21.]
 [ 6. 21. 47.  0. 15.]
 [ 7. 17. 63.  0. 19.]
 [ 8. 31. 62.  0. 23.]
 [ 9. 52. 33.  0. 11.]
 [10. 51. 21.  0.  5.]
 [11. 42. 41.  0. 19.]
 [12. 31. 32.  0. 29.]
 [13.  5. 25.  0. 23.]
 [14. 12. 42.  0. 21.]
 [15. 36. 16.  0. 10.]
 [16. 52. 41.  0. 15.]
 [17. 27. 23.  0.  3.]
 [18. 17. 33.  0. 41.]
 [19. 13. 13.  0.  9.]
 [20. 57. 58.  0. 28.]
 [21. 62. 42.  0.  8.]
 [22. 42. 57.  0.  8.]
 [23. 16. 57.  0. 16.]
 [24.  8. 52.  0. 10.]
 [25.  7. 38.  0. 28.]
 [26. 27. 68.  0.  7.]
 [27. 30. 48.  0. 15.]
 [28. 43. 67.  0. 14.]
 [29. 58. 48.  0.  6.]
 [30. 58. 27.  0. 19.]
 [31. 37. 69.  0. 11.]
 [32. 38. 46.  0. 12.]
 [33. 46. 10.  0. 23.]
 [34. 61. 33.  0. 26.]
 [35. 62. 63.  0. 17.]
 [36. 63. 69.  0.  6.]
 [37. 32. 22.  0.  9.]
 [38. 45. 35.  0. 15.]
 [39. 59. 15.  0. 14.]
 [40.  5. 

In [22]:
np.random.seed(0)

In [71]:
class Customer:
    def __init__(self,id,x,y,duration,demand):
        self.id = id
        self.x = x
        self.y = y 
        self.duration = duration
        self.demand = demand
        
            
    def __repr__(self):
        return "Client: " +str(int(self.id)) +" "+ str(int(self.x)) +" "+ str(int(self.y)) + " " + str(int(self.duration)) + " " + str(int(self.demand))

In [72]:
class Route: #aka chromosome
    def __init__(self, ID, depot):
        self.id = ID
        self.depot = depot #depot assigned to this route
        self.customers = list() 
        self.capacity = 0 #total of the demand
        self.ttlDistance = 0 #Ttl distance of the road
        
    def addDemand(self, d):
        self.capacity = self.capacity + d

    def addCustomer(self,c):
        self.customers.append(c)
        #print(str(self.id)+ " " +str(self.customers) + str(len(self.customers)))
        
    #getting total distance/cost of the route
    def calculTotalDistance(self):
        #print("Route:" + str(self.id)+ " customers:" + str(len(self.customers)))
        ttl = 0
        vTotal = 0
        if len(self.customers) == 0:
            return 0
        first_cust = self.customers[0]
        last_cust = self.customers[-1]
        #print(first_cust)
        #print(last_cust)
        #calculating cost between each cust
        for c in range(len(self.customers)-1):
            #print("x: " +str(self.customers[c].x) +" y: " + str(self.customers[c].y))
            ttl = ttl + Distance(self.customers[c].x,self.customers[c].y,self.customers[c+1].x,self.customers[c+1].y)
            pass
        #and between depot at the begining and end of the route since the veh needs to stop where it started
        for d in depotz:
            #get the right depot
            if (d[0]-n)-1 == self.depot:
                addtt = 0
                addtt = addtt + Distance(first_cust.x, first_cust.y, d[1], d[2])
                addtt = addtt + Distance(last_cust.x, last_cust.y, d[1], d[2])
                vTotal = ttl + addtt
                #print("Distance bw cust: " + str(ttl) + " Added to beg and end: " + str(addtt)+ " Total: " + str(vTotal))
        self.ttlDistance = vTotal
        return vTotal
        
        
    def __repr__(self):
        return "Route: " +str(self.id) +" "+ str(self.depot) + " " + str(len(self.customers))+ " Sz:"+ str(len(self.customers)) + " C:" + str(self.capacity) + " T:" + str(self.ttlDistance)

In [229]:
class Depot:
    def __init__(self, id, x, y): 
        self.id = id
        self.x = x
        self.y = y
        self.customers = list()
        self.maxVehicle = m
        self.routes = list()
        for i in range(self.maxVehicle):
            self.routes.append(Route(i, self.id))
        
    def addCustomer(self,c):
        #Selecting which route to assign it to
        np.random.randint(0,self.maxVehicle)
    
    def __repr__(self):
        return "Depot: " + str(self.id) +" "  + str(self.x) +" "+ str(self.y) + " "+ str(len(self.routes) )

In [230]:
def createIndividual():
    pass

def createSolution():
    pass
    
def makeGeneration(nbSolutions=indivPerGen):
    solutions = list()
    for i in range(nbSolutions):
        solutions.append(createSolution())
    return solutions

In [231]:
def makeSolution(depots):
    for client in clients:
        bestDist = 99999
        #assigning to closest depot
        closestDepot = None
        for depot in depots:
            distToDepot = Distance(depot.x, depot.y, client.x, client.y)
            if distToDepot < bestDist:
                bestDist = distToDepot
                closestDepot = depot
        closestDepot.addCustomer(client)
    return depots

In [232]:
def mainProg(epochs=1, indivPerGen=1):
    depots = list()
    clients = list()
    individuals = list()
    for depot in depotz:
        depots.append(Depot(depot[0], depot[1], depot[2])) 
    print(depots)
    for client in customerz:
        clients.append(Customer(client[0], client[1], client[2], client[3], client[4]))
    print(clients)

    
    #Epoch is a generation
    #First generation
    for ipg in range(indivPerGen):
        individuals.append(makeSolution(depots))
    print(individuals)
    """print(len(depots[0].customers))
    print(len(depots[1].customers))
    print(len(depots[2].customers))
    print(len(depots[3].customers))"""
    #earch generation
    for epoch in range(epochs):
        print("Generation: {}".format(epoch))
            
                
        
        

In [233]:
mainProg()

[Depot: 51.0 20.0 20.0 4, Depot: 52.0 30.0 40.0 4, Depot: 53.0 50.0 30.0 4, Depot: 54.0 60.0 50.0 4]
[Client: 1 37 52 0 7, Client: 2 49 49 0 30, Client: 3 52 64 0 16, Client: 4 20 26 0 9, Client: 5 40 30 0 21, Client: 6 21 47 0 15, Client: 7 17 63 0 19, Client: 8 31 62 0 23, Client: 9 52 33 0 11, Client: 10 51 21 0 5, Client: 11 42 41 0 19, Client: 12 31 32 0 29, Client: 13 5 25 0 23, Client: 14 12 42 0 21, Client: 15 36 16 0 10, Client: 16 52 41 0 15, Client: 17 27 23 0 3, Client: 18 17 33 0 41, Client: 19 13 13 0 9, Client: 20 57 58 0 28, Client: 21 62 42 0 8, Client: 22 42 57 0 8, Client: 23 16 57 0 16, Client: 24 8 52 0 10, Client: 25 7 38 0 28, Client: 26 27 68 0 7, Client: 27 30 48 0 15, Client: 28 43 67 0 14, Client: 29 58 48 0 6, Client: 30 58 27 0 19, Client: 31 37 69 0 11, Client: 32 38 46 0 12, Client: 33 46 10 0 23, Client: 34 61 33 0 26, Client: 35 62 63 0 17, Client: 36 63 69 0 6, Client: 37 32 22 0 9, Client: 38 45 35 0 15, Client: 39 59 15 0 14, Client: 40 5 6 0 7, Clie